# Bir Hayvan Uzman Sistemi Uygulamak

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) örneğinden.

Bu örnekte, bazı fiziksel özelliklere dayanarak bir hayvanı belirlemek için basit bir bilgi tabanlı sistem uygulayacağız. Sistem, aşağıdaki VE-VEYA ağacı ile temsil edilebilir (bu, tüm ağacın bir parçasıdır, kolayca bazı ek kurallar ekleyebiliriz):

![](../../../../../../translated_images/tr/AND-OR-Tree.5592d2c70187f283.webp)


## Geriye dönük çıkarımlı kendi uzman sistem kabuğumuz

Üretim kurallarına dayalı bilgi temsil için basit bir dil tanımlamaya çalışalım. Kuralları tanımlamak için anahtar kelimeler olarak Python sınıflarını kullanacağız. Temelde 3 tür sınıf olacaktır:
* `Ask`, kullanıcıya sorulması gereken bir soruyu temsil eder. Olası cevaplar kümesini içerir.
* `If`, bir kuralı temsil eder ve kuralın içeriğini saklamak için sadece sentaktik şeker gibidir.
* `AND`/`OR`, ağacın AND/OR dallarını temsil eden sınıflardır. İçerideki argüman listesini saklarlar. Kodu basitleştirmek için, tüm işlevsellik üst sınıf `Content` içinde tanımlanmıştır.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Sistemimizde, çalışma belleği **özellik-değer çiftleri** olarak **gerçekler** listesini içerir. Bilgi tabanı, eylemleri (çalışma belleğine eklenmesi gereken yeni gerçekler) AND-OR ifadeleriyle ifade edilen koşullara eşleyen büyük bir sözlük olarak tanımlanabilir. Ayrıca, bazı gerçekler `Ask` ile sorulabilir.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Geriye doğru çıkarım yapmak için, `Knowledgebase` sınıfını tanımlayacağız. İçerecek:
* Çalışan `memory` - nitelikleri değerlere eşleyen bir sözlük
* Yukarıda tanımlandığı biçimdeki Bilgi tabanı `rules`

İki ana yöntem şunlardır:
* Gerekirse çıkarım yaparak bir niteliğin değerini elde etmek için `get`. Örneğin, `get('color')` bir renk yuvasının değerini alır (gerekirse sorar ve bu değeri daha sonra kullanım için çalışan belleğe kaydeder). `get('color:blue')` derseniz, renk sorar ve ardından renge bağlı olarak `y`/`n` değerini döndürür.
* `eval` gerçek çıkarımı gerçekleştirir, yani VE/VEYA ağacında gezinir, alt hedefleri değerlendirir vb.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Şimdi hayvan bilgi tabanımızı tanımlayalım ve danışmanlığı gerçekleştirelim. Unutmayın, bu çağrı size sorular soracak. Evet-hayır sorularına `y`/`n` yazarak, daha uzun çoktan seçmeli sorulara ise 0..N arasında bir sayı belirterek cevap verebilirsiniz.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## İleri Çıkarım için Experta Kullanımı

Bir sonraki örnekte, bilgi temsili için kullanılan kütüphanelerden biri olan [Experta](https://github.com/nilp0inter/experta) kullanarak ileri çıkarımı uygulamaya çalışacağız. **Experta**, Python'da ileri çıkarım sistemleri oluşturmak için geliştirilmiş bir kütüphanedir ve klasik eski sistem [CLIPS](http://www.clipsrules.net/index.html) ile benzer olacak şekilde tasarlanmıştır.

İleri zincirlemeyi kendimiz de çok sorun yaşamadan uygulayabilirdik, ancak sezgisel (naif) uygulamalar genellikle çok verimli olmaz. Daha etkili kural eşleştirme için özel bir algoritma [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) kullanılır.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Sistemimizi `KnowledgeEngine` sınıfından türeyen bir sınıf olarak tanımlayacağız. Her kural, ne zaman tetikleneceğini belirten `@Rule` anotasyonuyla ayrı bir fonksiyon olarak tanımlanır. Kuralın içinde, `declare` fonksiyonunu kullanarak yeni gerçekler ekleyebiliriz ve bu gerçeklerin eklenmesi, ileri çıkarım motoru tarafından bazı ek kuralların çağrılmasına yol açar.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Bir bilgi tabanı tanımladıktan sonra, çalışma belleğimizi bazı başlangıç gerçekleriyle doldururuz ve sonra çıkarım yapmak için `run()` metodunu çağırırız. Sonuç olarak, çalışma belleğine yeni çıkarılan gerçeklerin eklendiğini görebilirsiniz; bunlar hayvanla ilgili nihai gerçeği de içerir (eğer tüm başlangıç gerçeklerini doğru şekilde ayarladıysak).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, yapay zeka çeviri servisi [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için özen gösterilse de, otomatik çevirilerin hatalar veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belge, kendi dilinde yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımı sonucu oluşabilecek yanlış anlamalar veya yorum hatalarından sorumlu değiliz.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
